In [1]:
import pandas as pd
import json, time, os
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
#Folder path must end with /
FOLDER = "/Users/echo/Documents/0424_Data_Enrichment/Project-3/Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['test.text']

# # Credentials and Accessing the API

In [2]:
with open("/Users/echo/Documents/0424_Data_Enrichment/.secret/tmdb_api.json", "r") as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()    
tmdb.API_KEY =  login['api-key']

In [3]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [4]:
def get_movie_with_rating(movie_id):
    #Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    #Save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    #Loop thru countries in releases
    for c in releases["countries"]:
        #if the country abbreviation == US
        if c["iso_3166_1"] == "US":
            #Save a certification key to new column
            info["certification"] = c["certification"]
    return info

# Confirm Your API Function works

In [5]:
#Getting info on "The Notebook" movie
get_movie_with_rating("tt0332280")

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 66.459,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/5ThIuO93vsk47oexKTSdfKEr7EC.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [6]:
#Getting info on "The Avengers" movie
get_movie_with_rating("tt0848228")

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 122.556,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

# Load in the Title Basics data

In [9]:
import pandas as pd
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv("/Users/echo/Documents/0424_Data_Enrichment/Project-3/Data/title_basics.csv.gz")
basics.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/echo/Documents/0424_Data_Enrichment/Project-3/Data/title_basics.csv.gz'

# Create Required Lists for the Loop
###  Define a list of the Years to Extract from the API

In [ ]:
YEARS_TO_GET = [2000,2001]

### Define an errors list

In [ ]:
#Create empty list to append errors later on 
errors = []

# Start OUTER loop

The progress bar works within the for statement of the for loop. Note that this will iterate through each year that is defined in the YEARS_TO_GET variable.

In [ ]:
# Start of OUTER loop
#progress bar based on years_to_get. DESC means Description name
for YEAR in tqdm_notebook(YEARS_TO_GET, desc="YEARS", position=0):
    #Defining the JSON file to store results for each year
    JSON_FILE = f"{FOLDER}tmdp_api_results {YEAR}.json"
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE, "w") as f:
            json.dump([{'imdb_id':0}],f)
    #Saving new year as the current df
    df = basics.loc[basics["startYear"] == YEAR].copy()
    # saving movie ids to list
    movie_ids = df["tconst"].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df["imdb_id"])]
    
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

                
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                        compression="gzip", index=False)
print(f"- Total errors: {len(errors)}")   

# Exploratory Data Analysis


In [ ]:
#Reading in 2000 movies from above output
results00 = pd.read_csv("/Users/echo/Documents/0424_Data_Enrichment/Project-3/Data/ final_tmdb_data_2000.csv.gz")
results00

In [ ]:
#Reading in 2001 movies from above output
results01 = pd.read_csv("/Users/echo/Documents/0424_Data_Enrichment/Project-3/Data/ final_tmdb_data_2001.csv.gz")
results01

In [ ]:
#Concatenate the data into 1 dataframe for the remainder of the analysis.
df = pd.concat([results00, results01], ignore_index=True)
df.info()

1. How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.

In [ ]:
#Filter for movies that made money.
valid_movies = df[(df["budget"] > 0) | (df["revenue"] > 0)]
valid_movies = valid_movies.shape[0]
valid_movies